In [30]:
import numpy as np
import os
import pandas as pd

import math

from datetime import datetime, timedelta
from dotenv import load_dotenv
 
import yfinance as yf

import hvplot.pandas

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [31]:
load_dotenv()

True

In [32]:
source_df = None

In [33]:
symbol = 'TSLA'
file_name = f"{symbol}.csv"
source_df = None

In [34]:
if os.path.exists(file_name):
    source_df = pd.read_csv(file_name)

In [35]:
if source_df is None:
    display("download")
    ticker = yf.Ticker(symbol)
    start = datetime.utcnow() - timedelta(days=10*365)
    end=datetime.utcnow() - timedelta(days=1)
    source_df = ticker.history(start=start, end=end)
    source_df.to_csv(f"{symbol}.csv")

In [36]:
display(source_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
2510,2024-02-16 00:00:00-05:00,202.059998,203.169998,197.399994,199.949997,111173600,0.0,0.0
2511,2024-02-20 00:00:00-05:00,196.130005,198.600006,189.130005,193.759995,104545800,0.0,0.0
2512,2024-02-21 00:00:00-05:00,193.360001,199.440002,191.949997,194.770004,103844000,0.0,0.0
2513,2024-02-22 00:00:00-05:00,194.000000,198.320007,191.360001,197.410004,92739500,0.0,0.0
2514,2024-02-23 00:00:00-05:00,195.309998,197.570007,191.500000,191.970001,78670300,0.0,0.0


In [37]:
bars_df = source_df.copy()
bars_df["Next Close"] = bars_df["Close"].shift(-1)
bars_df["Next High"] = bars_df["High"].shift(-1)
bars_df["Next Low"] = bars_df["Low"].shift(-1)

In [38]:
display(bars_df.head(100))
display(bars_df.tail(100))

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,Next Low
0,2014-02-27 00:00:00-05:00,17.416668,17.459999,16.555332,16.836000,269187000,0.0,0.0,16.320667,16.845333,16.170000
1,2014-02-28 00:00:00-05:00,16.643333,16.845333,16.170000,16.320667,218847000,0.0,0.0,16.704000,16.776667,15.666000
2,2014-03-03 00:00:00-05:00,15.817333,16.776667,15.666000,16.704000,196339500,0.0,0.0,16.989332,17.333332,16.855333
3,2014-03-04 00:00:00-05:00,17.232000,17.333332,16.855333,16.989332,131184000,0.0,0.0,16.844000,17.132668,16.786667
4,2014-03-05 00:00:00-05:00,17.114668,17.132668,16.786667,16.844000,89035500,0.0,0.0,16.862667,17.166668,16.629999
...,...,...,...,...,...,...,...,...,...,...,...
95,2014-07-15 00:00:00-04:00,15.115333,15.176667,14.540000,14.638667,85777500,0.0,0.0,14.477333,14.986667,14.454667
96,2014-07-16 00:00:00-04:00,14.788000,14.986667,14.454667,14.477333,60667500,0.0,0.0,14.360000,14.703333,14.240000
97,2014-07-17 00:00:00-04:00,14.410667,14.703333,14.240000,14.360000,69741000,0.0,0.0,14.668000,14.747333,14.395333
98,2014-07-18 00:00:00-04:00,14.396667,14.747333,14.395333,14.668000,63805500,0.0,0.0,14.702667,14.880667,14.448000


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,Next Low
2415,2023-10-02 00:00:00-04:00,244.809998,254.279999,242.619995,251.600006,123810400,0.0,0.0,246.529999,250.020004,244.449997
2416,2023-10-03 00:00:00-04:00,248.610001,250.020004,244.449997,246.529999,101985300,0.0,0.0,261.160004,261.859985,247.600006
2417,2023-10-04 00:00:00-04:00,248.139999,261.859985,247.600006,261.160004,129721600,0.0,0.0,260.049988,263.600006,256.250000
2418,2023-10-05 00:00:00-04:00,260.000000,263.600006,256.250000,260.049988,119159200,0.0,0.0,260.529999,261.649994,250.649994
2419,2023-10-06 00:00:00-04:00,253.979996,261.649994,250.649994,260.529999,117947000,0.0,0.0,259.670013,261.359985,252.050003
...,...,...,...,...,...,...,...,...,...,...,...
2510,2024-02-16 00:00:00-05:00,202.059998,203.169998,197.399994,199.949997,111173600,0.0,0.0,193.759995,198.600006,189.130005
2511,2024-02-20 00:00:00-05:00,196.130005,198.600006,189.130005,193.759995,104545800,0.0,0.0,194.770004,199.440002,191.949997
2512,2024-02-21 00:00:00-05:00,193.360001,199.440002,191.949997,194.770004,103844000,0.0,0.0,197.410004,198.320007,191.360001
2513,2024-02-22 00:00:00-05:00,194.000000,198.320007,191.360001,197.410004,92739500,0.0,0.0,191.970001,197.570007,191.500000


In [39]:
# So we want to add the high, low, and open for the 15 minute and 60 minute window
period = 1
windows = [3, 12]

for window in windows:
    wp = window*period
    # compute the rolling high, low, open for a given window.  the close is the same at this time for all windows
    bars_df[f"High - {wp}"] = bars_df["High"].rolling(window=window).max()
    bars_df[f"Low - {wp}"] = bars_df["Low"].rolling(window=window).min()
    bars_df[f"Open - {wp}"] = bars_df["Open"].shift(periods=window-1)

bars_df = bars_df.dropna()
display(bars_df)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,Next Low,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12
11,2014-03-14 00:00:00-04:00,15.686000,15.796000,15.221333,15.398000,124345500,0.0,0.0,15.598667,15.862000,15.366667,16.500000,15.221333,15.433333,17.459999,15.221333,17.416668
12,2014-03-17 00:00:00-04:00,15.663333,15.862000,15.366667,15.598667,88689000,0.0,0.0,16.002666,16.100000,15.668000,16.279333,15.221333,16.252666,17.333332,15.221333,16.643333
13,2014-03-18 00:00:00-04:00,15.796667,16.100000,15.668000,16.002666,93634500,0.0,0.0,15.722667,16.103333,15.567333,16.100000,15.221333,15.686000,17.333332,15.221333,15.817333
14,2014-03-19 00:00:00-04:00,16.092667,16.103333,15.567333,15.722667,76069500,0.0,0.0,15.660667,15.950000,15.557333,16.103333,15.366667,15.663333,17.333332,15.221333,17.232000
15,2014-03-20 00:00:00-04:00,15.744000,15.950000,15.557333,15.660667,57268500,0.0,0.0,15.259333,15.746667,15.166667,16.103333,15.557333,15.796667,17.166668,15.221333,17.114668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2509,2024-02-15 00:00:00-05:00,189.160004,200.880005,188.860001,200.449997,120831800,0.0,0.0,199.949997,203.169998,197.399994,200.880005,182.110001,183.990005,200.880005,175.009995,187.000000
2510,2024-02-16 00:00:00-05:00,202.059998,203.169998,197.399994,199.949997,111173600,0.0,0.0,193.759995,198.600006,189.130005,203.169998,183.350006,185.300003,203.169998,175.009995,188.500000
2511,2024-02-20 00:00:00-05:00,196.130005,198.600006,189.130005,193.759995,104545800,0.0,0.0,194.770004,199.440002,191.949997,203.169998,188.860001,189.160004,203.169998,175.009995,185.039993
2512,2024-02-21 00:00:00-05:00,193.360001,199.440002,191.949997,194.770004,103844000,0.0,0.0,197.410004,198.320007,191.360001,203.169998,189.130005,202.059998,203.169998,175.009995,184.259995


In [40]:
df = bars_df.reset_index()
ys = {
    "Predicted High": df["Next High"], 
    "Predicted Low": df["Next Low"], 
    "Predicted Close": df["Next Close"],
}

X = df.drop(columns=["Next Close", "Next Low", "Next High", "Date"])
display(X)

,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12
0,11,15.686000,15.796000,15.221333,15.398000,124345500,0.0,0.0,16.500000,15.221333,15.433333,17.459999,15.221333,17.416668
1,12,15.663333,15.862000,15.366667,15.598667,88689000,0.0,0.0,16.279333,15.221333,16.252666,17.333332,15.221333,16.643333
2,13,15.796667,16.100000,15.668000,16.002666,93634500,0.0,0.0,16.100000,15.221333,15.686000,17.333332,15.221333,15.817333
3,14,16.092667,16.103333,15.567333,15.722667,76069500,0.0,0.0,16.103333,15.366667,15.663333,17.333332,15.221333,17.232000
4,15,15.744000,15.950000,15.557333,15.660667,57268500,0.0,0.0,16.103333,15.557333,15.796667,17.166668,15.221333,17.114668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,2509,189.160004,200.880005,188.860001,200.449997,120831800,0.0,0.0,200.880005,182.110001,183.990005,200.880005,175.009995,187.000000
2499,2510,202.059998,203.169998,197.399994,199.949997,111173600,0.0,0.0,203.169998,183.350006,185.300003,203.169998,175.009995,188.500000
2500,2511,196.130005,198.600006,189.130005,193.759995,104545800,0.0,0.0,203.169998,188.860001,189.160004,203.169998,175.009995,185.039993
2501,2512,193.360001,199.440002,191.949997,194.770004,103844000,0.0,0.0,203.169998,189.130005,202.059998,203.169998,175.009995,184.259995


In [41]:
for name, y in ys.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    model = LinearRegression()
    model.fit(X_train, y_train)
    predicted_y_values = model.predict(X)
    # Create a copy of the original data
    bars_df[name] = predicted_y_values
    bars_df

In [42]:
display(bars_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,Next Low,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12,Predicted High,Predicted Low,Predicted Close
2509,2024-02-15 00:00:00-05:00,189.160004,200.880005,188.860001,200.449997,120831800,0.0,0.0,199.949997,203.169998,197.399994,200.880005,182.110001,183.990005,200.880005,175.009995,187.000000,205.024259,197.228928,200.086544
2510,2024-02-16 00:00:00-05:00,202.059998,203.169998,197.399994,199.949997,111173600,0.0,0.0,193.759995,198.600006,189.130005,203.169998,183.350006,185.300003,203.169998,175.009995,188.500000,203.443515,196.513618,200.201038
2511,2024-02-20 00:00:00-05:00,196.130005,198.600006,189.130005,193.759995,104545800,0.0,0.0,194.770004,199.440002,191.949997,203.169998,188.860001,189.160004,203.169998,175.009995,185.039993,197.239119,189.537807,193.829677
2512,2024-02-21 00:00:00-05:00,193.360001,199.440002,191.949997,194.770004,103844000,0.0,0.0,197.410004,198.320007,191.360001,203.169998,189.130005,202.059998,203.169998,175.009995,184.259995,199.521151,191.892347,195.891043
2513,2024-02-22 00:00:00-05:00,194.000000,198.320007,191.360001,197.410004,92739500,0.0,0.0,191.970001,197.570007,191.500000,199.440002,189.130005,196.130005,203.169998,177.110001,177.210007,200.876833,194.154974,197.514585


In [43]:
bars_df["High/Low Success"] = np.where(
    (bars_df["Next Close"] >= bars_df["Predicted Low"]),
    np.where(
        bars_df["Next Close"] <= bars_df["Predicted High"], 1, 0
    ), 0
)

In [44]:
bars_df["Predicted Close Delta"] = bars_df["Predicted Close"] - bars_df["Close"]
bars_df["Next Close Delta"] = bars_df["Next Close"] - bars_df["Close"]
bars_df["Close Product"] = bars_df["Next Close Delta"] * bars_df["Predicted Close Delta"]
bars_df["Close Success"] = np.where(
    bars_df["Close Product"] >= 0, 1, 0
)

In [45]:
actual_close = bars_df.hvplot.line(
    x="Date",
    y="Next Close",
)

In [46]:
predicted_high = bars_df.hvplot.line(
    x="Date",
    y="Predicted High",
)

In [47]:
predicted_low = bars_df.hvplot.line(
    x="Date",
    y="Predicted Low",
)

In [48]:
predicted_close = bars_df.hvplot.line(
    x="Date",
    y="Predicted Close",
)

In [49]:
actual_close * predicted_high * predicted_low * predicted_close

:Overlay
   .Curve.I   :Curve   [Date]   (Next Close)
   .Curve.II  :Curve   [Date]   (Predicted High)
   .Curve.III :Curve   [Date]   (Predicted Low)
   .Curve.IV  :Curve   [Date]   (Predicted Close)

In [50]:
bars_df.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,Next Low,...,Low - 12,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success
count,2503.000000,2503.000000,2503.000000,2503.000000,2.503000e+03,2503.0,2503.000000,2503.000000,2503.000000,2503.000000,...,2503.000000,2503.000000,2503.000000,2503.000000,2503.000000,2503.000000,2503.000000,2503.000000,2503.000000,2503.000000
mean,97.291351,99.418204,94.996162,97.262492,1.124275e+08,0.0,0.003196,97.333036,99.490826,95.066589,...,86.979631,96.523549,99.525508,95.054693,97.335906,0.510987,0.073415,0.070544,0.283638,0.508989
std,109.088418,111.508717,106.410047,108.991297,7.450664e+07,0.0,0.116529,108.995432,111.513427,106.415561,...,97.498036,109.038914,111.576409,106.320011,108.934031,0.499979,0.595391,5.483153,6.763023,0.500019
min,9.488000,10.331333,9.403333,9.578000,1.062000e+07,0.0,0.000000,9.578000,10.331333,9.403333,...,9.403333,9.488000,9.872898,9.080079,9.493182,0.000000,-8.798350,-46.480011,-149.244218,0.000000
25%,16.063001,16.276667,15.741000,16.078667,6.628275e+07,0.0,0.000000,16.081667,16.290000,15.755000,...,14.756000,16.033000,16.369088,15.819810,16.136294,0.000000,-0.090739,-0.478001,-0.080133,0.000000
50%,22.366667,22.816000,22.000000,22.337999,9.315600e+07,0.0,0.000000,22.340000,22.818666,22.000000,...,20.235332,22.210667,22.771267,22.221179,22.596231,1.000000,0.110896,0.025333,0.000317,1.000000
75%,203.974998,208.811661,198.580002,204.459999,1.313042e+08,0.0,0.000000,204.459999,208.811661,198.580002,...,182.326660,203.974998,208.941476,199.073198,203.965065,1.000000,0.279292,0.572000,0.112439,1.000000
max,411.470001,414.496674,405.666656,409.970001,9.140820e+08,0.0,5.000000,409.970001,414.496674,405.666656,...,334.059998,411.470001,422.467615,400.501669,412.242139,1.000000,3.210933,47.666656,80.199884,1.000000


In [51]:
bars_df["Action"] = np.where(
    bars_df["Next Close"] > bars_df["Close"], 1, -1
)

In [52]:
display(bars_df.head())
display(bars_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,...,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success,Action
11,2014-03-14 00:00:00-04:00,15.686000,15.796000,15.221333,15.398000,124345500,0.0,0.0,15.598667,15.862000,...,17.416668,15.621195,14.676951,15.130649,1,-0.267351,0.200667,-0.053649,0,1
12,2014-03-17 00:00:00-04:00,15.663333,15.862000,15.366667,15.598667,88689000,0.0,0.0,16.002666,16.100000,...,16.643333,15.871677,15.004580,15.407022,0,-0.191645,0.403999,-0.077424,0,1
13,2014-03-18 00:00:00-04:00,15.796667,16.100000,15.668000,16.002666,93634500,0.0,0.0,15.722667,16.103333,...,15.817333,16.248875,15.424790,15.765771,1,-0.236896,-0.280000,0.066331,1,-1
14,2014-03-19 00:00:00-04:00,16.092667,16.103333,15.567333,15.722667,76069500,0.0,0.0,15.660667,15.950000,...,17.232000,15.936775,15.106619,15.496033,1,-0.226634,-0.061999,0.014051,1,-1
15,2014-03-20 00:00:00-04:00,15.744000,15.950000,15.557333,15.660667,57268500,0.0,0.0,15.259333,15.746667,...,17.114668,15.931486,15.142630,15.494065,1,-0.166602,-0.401335,0.066863,1,-1


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,...,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success,Action
2509,2024-02-15 00:00:00-05:00,189.160004,200.880005,188.860001,200.449997,120831800,0.0,0.0,199.949997,203.169998,...,187.000000,205.024259,197.228928,200.086544,1,-0.363453,-0.500000,0.181727,1,-1
2510,2024-02-16 00:00:00-05:00,202.059998,203.169998,197.399994,199.949997,111173600,0.0,0.0,193.759995,198.600006,...,188.500000,203.443515,196.513618,200.201038,0,0.251041,-6.190002,-1.553945,0,-1
2511,2024-02-20 00:00:00-05:00,196.130005,198.600006,189.130005,193.759995,104545800,0.0,0.0,194.770004,199.440002,...,185.039993,197.239119,189.537807,193.829677,1,0.069682,1.010010,0.070380,1,1
2512,2024-02-21 00:00:00-05:00,193.360001,199.440002,191.949997,194.770004,103844000,0.0,0.0,197.410004,198.320007,...,184.259995,199.521151,191.892347,195.891043,1,1.121039,2.639999,2.959541,1,1
2513,2024-02-22 00:00:00-05:00,194.000000,198.320007,191.360001,197.410004,92739500,0.0,0.0,191.970001,197.570007,...,177.210007,200.876833,194.154974,197.514585,0,0.104581,-5.440002,-0.568923,0,-1


In [53]:
current_position = 0
starting_cash = 1000
current_cash = starting_cash
max_position = 50
for index, row in bars_df.iterrows():
    # Get the current action
    close = row["Close"]
    action = row["Action"]

    # Take the action if possible
    if action > 0:
        if action + current_position <= max_position and action * close < current_cash:
            current_position += action
            current_cash -= action*close
        else:
            action = 0
    elif action < 0:
        if action + current_position >= 0:
            current_position += action
            current_cash += -action*close
        else:
            action = 0

    # Update enabled and position
    bars_df.loc[index, "Position"] = current_position
    bars_df.loc[index, "Cash"] = current_cash

In [54]:
# Compute Holdings
bars_df["Holdings"] = bars_df["Close"] * bars_df["Position"]

# Compute Profit
bars_df["Strategy Value"] = bars_df["Holdings"] + bars_df["Cash"]
starting_close = bars_df.iloc[0]["Close"] 
display(f"starting_close {starting_close}")
starting_shares = math.floor(starting_cash / starting_close)
display(f"starting_shares {starting_shares}")
bars_df["Stock Value"] =  bars_df["Close"] * starting_shares

# Compute Returns
bars_df["Stock Returns"] = bars_df["Close"].pct_change()
bars_df["Strategy Returns"] = bars_df["Strategy Value"].pct_change()

# Compute Cumulative Daily Returns
bars_df["Stock Cumulative Returns"] = (bars_df["Stock Returns"] + 1).cumprod()
bars_df["Strategy Cumulative Returns"] = (bars_df["Strategy Returns"] + 1).cumprod()

'starting_close 15.39799976348877'

'starting_shares 64'

In [55]:
bars_df.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Next Close,Next High,...,Action,Position,Cash,Holdings,Strategy Value,Stock Value,Stock Returns,Strategy Returns,Stock Cumulative Returns,Strategy Cumulative Returns
2509,2024-02-15 00:00:00-05:00,189.160004,200.880005,188.860001,200.449997,120831800,0.0,0.0,199.949997,203.169998,...,-1,40.0,3886.152648,8017.999878,11904.152526,12828.799805,0.062212,0.042138,13.017924,11.904153
2510,2024-02-16 00:00:00-05:00,202.059998,203.169998,197.399994,199.949997,111173600,0.0,0.0,193.759995,198.600006,...,-1,39.0,4086.102645,7798.049881,11884.152526,12796.799805,-0.002494,-0.001680,12.985453,11.884153
2511,2024-02-20 00:00:00-05:00,196.130005,198.600006,189.130005,193.759995,104545800,0.0,0.0,194.770004,199.440002,...,1,40.0,3892.342650,7750.399780,11642.742431,12400.639648,-0.030958,-0.020314,12.583452,11.642742
2512,2024-02-21 00:00:00-05:00,193.360001,199.440002,191.949997,194.770004,103844000,0.0,0.0,197.410004,198.320007,...,1,41.0,3697.572646,7985.570175,11683.142821,12465.280273,0.005213,0.003470,12.649046,11.683143
2513,2024-02-22 00:00:00-05:00,194.000000,198.320007,191.360001,197.410004,92739500,0.0,0.0,191.970001,197.570007,...,-1,40.0,3894.982650,7896.400146,11791.382796,12634.240234,0.013554,0.009265,12.820497,11.791383


In [56]:
returns = bars_df.hvplot.line(
    x="Date",
    y=["Strategy Cumulative Returns", "Stock Cumulative Returns"],
).opts(
    title=f"Stock vs Strategy Returns for {symbol} using Linear Regression",
    ylabel="Percent Returns"
)
returns

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [57]:
portfolio = bars_df.hvplot.line(
    x="Date",
    y=["Cash", "Holdings"],
).opts(
    title=f"Cash vs Holdings for {symbol} using Linear Regression",
    ylabel="Dollar Value"
)
portfolio

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [58]:
position = bars_df.hvplot.line(
    x="Date",
    y=["Position"],
).opts(
    title=f"Number of shares of {symbol} using Linear Regression",
    ylabel="Dollar Value"
)
position

:Curve   [Date]   (Position)